In [2]:
import numpy as np
import quadtrees as qtrees

In [37]:
#BARNES HUT
#idea is simple: given particle system with interations,
#sum up the interactions on one particular particle

def SumTraverse(root,loc,m,interFun,farParameter=2):
    #root is QTree root
    #loc is location of particle
    #m is its mass
    #interFun is interactionFunction(m,m0,x,x0) between two particles
    #farParameter=2 means to approximate if dist/boxsize > 2
    #find all interactions
    
    if(len(root.children)>0):
        inter=0
        for child in root.children:
            
            if( np.linalg.norm(child.coord-loc,2)/child.boxsize >farParameter ):
                #if it can be approximated,then
                print(loc.shape,child.coord.shape)
                
                inter=inter+interFun(m,child.mass,loc,child.coord)
                print(inter)
            else:
                #if cannot be approximated
                inter=inter+SumTraverse(child,loc,m,interFun,farParameter)
                
        return inter
    else:
        return interFun(m,root.mass,loc,root.coord)
        
        
def BHSummation(xs,ms,interFun,farParameter=2):
    # farParameter = 2  if distance is 2 times larger than box size, then approximate

    treeRoot = qtrees.TreeNode()
    treeRoot = qtrees.ConstructTree(xs,ms,treeRoot,level=0)
    
    inters = np.zeros([xs.shape[1]])
    for i in np.arange(xs.shape[1]):
        inters[i] = SumTraverse(treeRoot,xs[:,i],ms[i],interFun,farParameter)
    return inters

In [38]:
def GravInteraction(m0,m1,x0,x1):
    return m0*m1/np.linalg.norm(x1-x0,2,axis=0)

GravInteraction(1,2,np.array([1,2]),np.array([2,2]))

(2,)


In [39]:
xs = np.array([[1,1.5,-1,-0.8],[0.9,-0.5,-0.3,0.7]])*100
ms= np.array([1,2,1,2])

treeRoot = qtrees.TreeNode()#root
treeRoot=qtrees.ConstructTree(xs,ms,treeRoot,level=0)

In [34]:
BHSummation(xs,ms,GravInteraction,farParameter=2)

(2,) (2, 1)
[0.00419222 0.00444994]


ValueError: setting an array element with a sequence.

In [ ]:
GravInteraction(ms[:,np.newaxis],ms[np.newaxis,:],\
                np.kron(x0,np.ones([1,4])),np.kron(np.ones([1,4]),x1)) @ np.ones([4])